In [1]:
# Import libraries.
import json
from pathlib import Path
from collections import namedtuple

from dataclasses import replace
import pandas as pd
import phoenix as px

/home/avila/micromamba/envs/phoenix/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
PATH_TRAIN="/data/bacteria/experiments/autoencoders/6mer/27122023-5"
KMER=6
feature_pred_label = "consensus_10"
xq = np.load(f"{PATH_TRAIN}/faiss-embeddings/query_embeddings.npy").astype("float32")

In [3]:
df_labels = pd.read_csv(f"{PATH_TRAIN}/test/test_index.tsv",sep="\t", usecols=["GT",feature_pred_label])

In [4]:
Phoenix=namedtuple("Phoenix",["id","path_img","link_img","label","vector_img"])
data_phoenix = []

with open(f"{PATH_TRAIN}/faiss-embeddings/id_query_embeddings.json") as fp:
    id_query = json.load(fp)
    id_query = {int(k): v for k,v in id_query.items()}

REPLACE="/data/bacteria/experiments/autoencoders"
for idx, path_npy in id_query.items():
    _id = Path(path_npy).stem
    path_img = path_npy.replace("fcgr","img").replace(".npy",".jpeg")
    link_img = path_img.replace(REPLACE,"http://localhost:6007") # I choose the port 6007, and I need to run `python -m http.server 6007` in the REPLACE directory
    label = Path(path_img).parent.stem.split("__")[0]
    embedding = xq[idx]
    data_phoenix.append(
        Phoenix(_id, path_img, link_img, label, embedding)
    )
    

In [5]:
test_df = pd.concat([pd.DataFrame(data_phoenix), df_labels],axis=1)
test_df.head()

,id,path_img,link_img,label,vector_img,GT,consensus_10
0,SAMEA1030444,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/escherichia_col...,escherichia_coli,"[0.0, 0.0, 0.0087590385, 0.05582408, 0.0021745...",escherichia_coli,escherichia_coli
1,SAMEA3373505,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/neisseria_gonor...,neisseria_gonorrhoeae,"[0.0, 0.0, 0.08142213, 0.0, 0.0, 0.0, 0.0, 0.1...",neisseria_gonorrhoeae,neisseria_gonorrhoeae
2,SAMEA104093527,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/staphylococcus_...,staphylococcus_aureus,"[0.0, 0.088135384, 0.09180802, 0.0, 0.0, 0.0, ...",staphylococcus_aureus,staphylococcus_aureus
3,SAMN03074833,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/klebsiella_aero...,klebsiella_aerogenes,"[0.0, 0.0, 0.032542888, 0.0, 0.010286896, 0.0,...",klebsiella_aerogenes,klebsiella_pneumoniae
4,SAMEA2159852,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/streptococcus_p...,streptococcus_pyogenes,"[0.0, 0.087676406, 0.039066393, 0.0, 0.0, 0.0,...",streptococcus_pyogenes,streptococcus_pyogenes


In [6]:
# Define schemas that tell Phoenix which columns of your DataFrames correspond to features, predictions, actuals (i.e., ground truth), embeddings, etc.
test_schema = px.Schema(
    prediction_id_column_name="id",
    # timestamp_column_name="prediction_ts",
    prediction_label_column_name=feature_pred_label,
    actual_label_column_name="GT",
    embedding_feature_column_names={
        "image_embedding": px.EmbeddingColumnNames(
            vector_column_name="vector_img",
            link_to_data_column_name="link_img",
        ),
    },
)

In [7]:
labels = ['bacillus_anthracis',
 'bacillus_cereus',
 'bacillus_thuringiensis',
 'brucella_abortus',
 'brucella_melitensis',
 'brucella_suis',
 'burkholderia_gladioli',
 'cutibacterium_acnes',
 'enterobacter_cloacae',
 'enterobacter_hormaechei',
 'escherichia_coli',
 'eubacterium_hallii',
 'klebsiella_pneumoniae',
 'lactobacillus_salivarius',
 'lactococcus_lactis',
 'mycobacterium_avium',
 'mycobacterium_chimaera',
 'mycobacterium_kansasii',
 'mycobacteroides_abscessus',
 'mycobacteroides_chelonae',
 'roseburia_hominis',
 'shigella_flexneri',
 'streptococcus_agalactiae',
 'streptococcus_mitis',
 'streptococcus_oralis',
 'streptococcus_sp_group_b']

In [8]:
# Define schemas that tell Phoenix which columns of your DataFrames correspond to features, predictions, actuals (i.e., ground truth), embeddings, etc.
# prod_schema = replace(train_schema, actual_label_column_name=None)

# Define your production and training datasets.
# prod_ds = px.Dataset(prod_df, prod_schema)
# train_ds = px.Dataset(train_df, train_schema)
print(test_df.query(f"GT in {labels}").shape)
test_ds = px.Dataset(test_df.query(f"GT in {labels}"), test_schema)

# Launch Phoenix.
session = px.launch_app(
    # prod_ds, 
    test_ds,
    port=6006
    )

# View the Phoenix UI in the browser
session.url
# px.active_session().view()

(2403, 7)
🌍 To view the Phoenix app in your browser, visit http://127.0.0.1:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


'http://127.0.0.1:6006/'

/home/avila/micromamba/envs/phoenix/lib/python3.11/site-packages/phoenix/server/api/types/EmbeddingDimension.py:422: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prediction_id=dataset[PREDICTION_ID][row_id],
/home/avila/micromamba/envs/phoenix/lib/python3.11/site-packages/phoenix/server/api/types/EmbeddingDimension.py:423: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  link_to_data=dataset[self.dimension.link_to_data][row_id],
/home/avila/micromamba/envs/phoenix/lib/python3.11/site-packages/phoenix/server/api/types/EmbeddingDimension.py:424: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. 